In [97]:
import csv
import re
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [100]:
## file name and setup 

file_name = 'index'

#pollutants
terms_of_interest = ['carcinogenic', 'carcinogens', 'arsenic', 'nitrate', 'nitrite', 'nitrates']

#urls
basic_url = 'https://www.communitywatercenter.org/news?n=448&page='
n_pages = 34

#cities
location_names = pd.read_csv('CA_names.csv')

In [99]:
pagelinks =[]

for number in range(1,n_pages+1):

    url = basic_url + str(number)
    page = urlopen(url)
    soup = BeautifulSoup(page, 'html.parser') #containing the HTML of the page
    name_box = soup.find_all('h3', attrs={'class': 'news_heading'})
    for count in range(0, len(name_box)):
            for link in name_box[count].find_all('a', href=True):
                pagelinks.append(link['href']) 

In [101]:
# For finding cities and counties

cities = location_names['city'].tolist() 
counties = location_names['county_name'].tolist()
cities.remove('August')
cities.remove('Challenge')
locations = cities + counties

def intersect(a, b):
     return list(set(a) & set(b))

In [102]:
#Writing to CSV file (basic)

for count in range(0, len(pagelinks)):
    
    page_url = "https://www.communitywatercenter.org" + pagelinks[count]
    page = urlopen(page_url)
    soup = BeautifulSoup(page, 'html.parser')
    
    title = soup.title
    date = soup.find_all(string=re.compile("Published on"))
    L1 = soup.get_text().split()
    city = intersect(L1, cities)
    county = intersect(L1, counties)
    
    with open(file_name + '.csv', 'a') as mycsv: ## a for append
        
        writer = csv.writer(mycsv)
        
        for term in range(0, len(terms_of_interest)):
        
            phrase = soup.find_all(string=re.compile(terms_of_interest[term]))
                    
            for item in range(0, len(phrase)):
                
                writer.writerow([phrase[item], str(date)[19:-8], city, county, title.string, page_url])